In [21]:
import os 
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Input, Lambda, Flatten, Reshape, Conv2D, Conv2DTranspose, UpSampling2D
from tensorflow.keras.layers import Dropout, BatchNormalization, LeakyReLU, concatenate, MaxPooling2D
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.optimizers import Adam
import numpy as np
from tensorflow.keras.callbacks  import ModelCheckpoint
from moviepy.editor import * 
from natsort import natsorted
from tqdm import tqdm
from IPython.display import clear_output

In [ ]:
#@title Colab
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#@title Local
os.chdir("E:/")

In [12]:
## раскадровка фильмов для сбора базы. Ссылка на собранную базу https://drive.google.com/file/d/1MQ36P2OpQDoO62_nqkk7uuuKlPIcdCd5/view?usp=sharing
"""
count = 0                         
imId = 'Лжец. лжец - 720HD - [ KinoMobi.net ].mp4'
cap= cv2.VideoCapture(imId)
frameRate = cap.get(5)*3
image_folder = 'forsort'
while cap.isOpened():
  frameId = cap.get(1)
  ret, frame = cap.read()
  if (ret != True):
    break
  if (frameId % math.floor(frameRate) == 0):
      filename ="frameF%d.jpg" % count;count+=1
      cv2.imwrite(image_folder+'/'+ filename, frame)
cap.release()
cv2.destroyAllWindows()

In [ ]:
!unzip -q '/content/drive/MyDrive/Базы/Jim.zip'

In [45]:
train_datagen = ImageDataGenerator(rescale=1./255, validation_split= 0.2, horizontal_flip=True, rotation_range=5)

train_generator = train_datagen.flow_from_directory(directory='/content/Jim', target_size=(224,224), subset='training', batch_size=64)
val_generator = train_datagen.flow_from_directory(directory='/content/Jim', target_size=(224,224), subset='validation', batch_size=64)

Found 9818 images belonging to 2 classes.
Found 2454 images belonging to 2 classes.


In [47]:
vgg_model = VGG16(include_top=False)
vggModel = Model(vgg_model.input,vgg_model.output)

inputs = Input(shape=(224,224,3))
popOut = vggModel(inputs)
flatten = Flatten()(popOut)
dense1 = Dense(1024, activation = 'relu')(flatten)
drop1 = Dropout(0.25)(dense1)
dense2 = Dense(1024, activation = 'relu')(drop1)
drop2 = Dropout(0.25)(dense2)
outputs = Dense(2, activation = 'softmax')(drop2)
model = Model(inputs, outputs)
vggModel.trainable = False

model.compile(optimizer="Adam", loss="categorical_crossentropy", metrics=['accuracy'])
filepath="best_model.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [ ]:
model.fit(train_generator,  epochs=10, validation_data=val_generator, callbacks = callbacks_list)

Epoch 1/10
154/154 [==============================] - 119s 767ms/step - loss: 0.8917 - accuracy: 0.7245 - val_loss: 0.5660 - val_accuracy: 0.7645

Epoch 00001: val_accuracy improved from -inf to 0.76447, saving model to best_model.h5
Epoch 2/10
154/154 [==============================] - 119s 776ms/step - loss: 0.3185 - accuracy: 0.8615 - val_loss: 0.6689 - val_accuracy: 0.7698

Epoch 00002: val_accuracy improved from 0.76447 to 0.76976, saving model to best_model.h5
Epoch 3/10
154/154 [==============================] - 116s 754ms/step - loss: 0.2543 - accuracy: 0.8914 - val_loss: 0.6014 - val_accuracy: 0.7689

Epoch 00003: val_accuracy did not improve from 0.76976
Epoch 4/10
154/154 [==============================] - 117s 758ms/step - loss: 0.2213 - accuracy: 0.9105 - val_loss: 0.5788 - val_accuracy: 0.7547

Epoch 00004: val_accuracy did not improve from 0.76976
Epoch 5/10
154/154 [==============================] - 119s 773ms/step - loss: 0.2279 - accuracy: 0.9028 - val_loss: 0.5980 - 

In [44]:
model.save('E:\content\drive\MyDrive\TrainedModels\jimVal80')

INFO:tensorflow:Assets written to: E:\content\drive\MyDrive\TrainedModels\jimVal80\assets


In [7]:
model.load_weights('best_model.h5')

In [41]:
def take_frames_from_video_by_seconds(model, video_path, divide_sec_on):
  second = 0 
  cap = cv2.VideoCapture(video_path)
  frameRate = cap.get(5)/divide_sec_on
  image_folder = 'temp'
  while cap.isOpened():
    frameId = cap.get(1)
    ret, frame = cap.read()
    if not ret:
      break
    if frameId%round(frameRate,0)==1:  
      filename =f"{round(second, 1)}.jpg";second+=(1/round(frameRate,0))
      cv2.imwrite(image_folder+'/'+ filename, cv2.resize(frame, (224,224)))
  cap.release()
  cv2.destroyAllWindows()
  return round(frameRate,0)
  
def predict_from_frames(frameRate, model=model, path='temp'):
  counter = 0
  predict = []
  temp = []
  for filename in tqdm(natsorted(os.listdir(path))):

      img = cv2.imread(path+'/'+filename)
      img = img/255.
      pred = model.predict(img[None, :,:,:])
      temp.append(np.argmax(pred[0]))
      counter+=1
      if counter>=frameRate:
        predict.append(temp)
        temp = []
        counter = 0
  return np.array(predict)

def get_timings_from_predict(predict, frameRate):
  seconds = []
  for s, pred in enumerate(predict):
    if sum(pred)<frameRate*0.4:
      seconds.append(s)
  return seconds

def preprocess_timings(list_of_seconds, bias): # bias - разница между планами. Не разрезать кусок видео, если между от джима до джима промежуток bias секунд(0 - полная нетерпимость к отсутствию джима)
  time_list = []
  temp = []
  for i in range(len(list_of_seconds)-1):
    temp.append(list_of_seconds[i])
    if list_of_seconds[i+1]-list_of_seconds[i]>bias+1:
      time_list.append([temp[0], temp[-1]+1])
      temp = []
  return time_list

def extract_samples(filepath, time_list):
  for timings in time_list:
    clip = VideoFileClip(filepath) 
    clip = clip.subclip(timings[0], timings[1]) 
    clip.write_videofile(filename = f"cut/{timings[0]}.mp4")
    clear_output()
    clip.close()

def make_one_clip_from_cuts(cutdir="cut/"):
  clips =[]
  for root, dirs, files in os.walk(cutdir):
      files = natsorted(files)
      for file in tqdm(files):
        filePath = os.path.join(root, file)
        video = VideoFileClip(filePath)
        clips.append(video)
  final_clip = concatenate_videoclips(clips)
  final_clip.write_videofile('output.mp4', logger=None, verbose=False)

In [42]:
def main(filepath,model=model, fps_division = 5, bias = 1):
  try: os.mkdir('temp'); os.mkdir('cut')
  except Exception:
    pass
  print('Подготовка к распознаванию')
  divided_fps = take_frames_from_video_by_seconds(model, filepath, fps_division)
  clear_output()
  print('Распознавание')
  predict = predict_from_frames(divided_fps)
  clear_output()
  print('Расчет длительности клипа')
  seconds = get_timings_from_predict(predict, divided_fps)
  time_list = preprocess_timings(seconds, bias)
  clear_output()
  print('Нарезка клипов')
  extract_samples(filepath, time_list)
  clear_output()
  print('Склеивание клипов. Расчетное время Джима в кадре:',round(len(seconds)/60,0), "минут", len(seconds)%60, 'секунд')
  make_one_clip_from_cuts()

In [43]:
main('Shou_Trumana_Kinosimka.RU (online-video-cutter.com).mp4')

  0%|                                                                                          | 0/127 [00:00<?, ?it/s]

Склеивание клипов. Расчетное время Джима в кадре: 9.0 минут 44 секунд


chunk:   0%|▎                                                            | 69/15090 [00:00<00:42, 351.49it/s, now=None]

Moviepy - Building video __temp__.mp4.
MoviePy - Writing audio in __temp__TEMP_MPY_wvf_snd.mp3


t:   0%|                                                                 | 28/16408 [00:00<00:59, 277.22it/s, now=None]

MoviePy - Done.
Moviepy - Writing video __temp__.mp4



Moviepy - Done !
Moviepy - video ready __temp__.mp4
